In [1]:
import argparse
import pandas as pd
from tqdm import tqdm
import yaml
import math
import numpy as np
from datetime import datetime
import json
import bisect

config = yaml.load(open('../../config/config.yaml'), Loader=yaml.FullLoader)


<ipython-input-1-a9149d4ca648>:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open('../../config/config.yaml'))


In [ ]:
config